# Multi-label Classification
for COMP47590: Advanced Machine Learning

Author(s): Arlene John and Vitoria Fahed

## Import Packages Etc

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import math
import random
import copy

from IPython.display import display, HTML, Image


from sklearn.metrics import hamming_loss,make_scorer, accuracy_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from sklearn import tree
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from scipy.spatial import distance
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain

# New Section

## Task 0: Load the Yeast Dataset

In [2]:
dataset = pd.read_csv('yeast.csv')
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0.004168,-0.170975,-0.156748,-0.142151,0.058781,0.026851,0.197719,0.041850,0.066938,-0.056617,...,0,0,1,1,0,0,0,1,1,0
1,-0.103956,0.011879,-0.098986,-0.054501,-0.007970,0.049113,-0.030580,-0.077933,-0.080529,-0.016267,...,0,0,0,0,0,0,0,0,0,0
2,0.509949,0.401709,0.293799,0.087714,0.011686,-0.006411,-0.006255,0.013646,-0.040666,-0.024447,...,0,0,0,0,0,0,0,1,1,0
3,0.119092,0.004412,-0.002262,0.072254,0.044512,-0.051467,0.074686,-0.007670,0.079438,0.062184,...,0,0,0,0,0,0,0,0,0,0
4,0.042037,0.007054,-0.069483,0.081015,-0.048207,0.089446,-0.004947,0.064456,-0.133387,0.068878,...,1,1,0,0,0,0,0,0,0,0


In [3]:
dataset.describe()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
count,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,...,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000,2417.000000
mean,0.001173,-0.000436,-0.000257,0.000265,0.001228,0.000475,0.001107,0.000420,0.001076,-0.000009,...,0.298717,0.247000,0.177079,0.198593,0.073645,0.104675,0.119570,0.751345,0.744311,0.014067
std,0.097411,0.097885,0.097746,0.096969,0.096909,0.097306,0.097170,0.096803,0.096326,0.096805,...,0.457790,0.431356,0.381815,0.399024,0.261246,0.306198,0.324525,0.432323,0.436338,0.117792
min,-0.371146,-0.472632,-0.339195,-0.467945,-0.367044,-0.509447,-0.319928,-0.594498,-0.369712,-0.767128,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.053655,-0.058734,-0.057526,-0.057149,-0.058461,-0.060212,-0.058445,-0.062849,-0.063472,-0.065010,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.003649,-0.003513,0.002892,-0.000153,0.005565,0.000321,0.006179,0.001436,0.003515,0.002432,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,0.057299,0.048047,0.061007,0.054522,0.066286,0.059908,0.068892,0.061418,0.064958,0.063096,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,0.520272,0.614114,0.353241,0.568960,0.307649,0.336971,0.351401,0.454591,0.419852,0.420876,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
features=dataset.iloc[:,0:103]
labels=dataset.iloc[:,103:117]

In [5]:
# Create a dataframe to store the values of the evaluation scores to better compare them
initialize = np.zeros(shape=6)
df = pd.DataFrame({'Binary Relavance (BR) without Undersampling':initialize , 'Binary Relavance with Undersampling':initialize , 'Optimised BR Model with F1 score (w/o undersampling)':initialize , 'Optimised BR Model with Accuracy (w/o undersampling)':initialize, 'Optimised BR Model with F1 score (undersampled)':initialize, 'Optimised BR Model with accuracy (undersampled)':initialize} , index=['Hamming Loss ValidSet','F1 Score ValidSet','Accuracy ValidSet', 'Hamming Loss TestSet', 'F1 Score TestSet', 'Accuracy TestSet'])
df

,Binary Relavance (BR) without Undersampling,Binary Relavance with Undersampling,Optimised BR Model with F1 score (w/o undersampling),Optimised BR Model with Accuracy (w/o undersampling),Optimised BR Model with F1 score (undersampled),Optimised BR Model with accuracy (undersampled)
Hamming Loss ValidSet,0.0,0.0,0.0,0.0,0.0,0.0
F1 Score ValidSet,0.0,0.0,0.0,0.0,0.0,0.0
Accuracy ValidSet,0.0,0.0,0.0,0.0,0.0,0.0
Hamming Loss TestSet,0.0,0.0,0.0,0.0,0.0,0.0
F1 Score TestSet,0.0,0.0,0.0,0.0,0.0,0.0
Accuracy TestSet,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Create a dataframe to store the values of the evaluation scores to compare Binary Relevance and Classifier Chains
initialize = np.zeros(shape=6)
df2 = pd.DataFrame({'Binary Relavance without Undersampling':initialize , 'Binary Relavance with Undersampling':initialize,'Optimal Binary Relavance without Undersampling (F1 score)':initialize , 'Optimal Binary Relavance with Undersampling (F1 score)':initialize, 'Classifier Chain (CC)':initialize,'Optimal Classifier Chains (F1 score)':initialize, 'Optimal Changed Chain Sequence CC':initialize} , index=['Hamming Loss ValidSet','F1 Score ValidSet','Accuracy ValidSet', 'Hamming Loss TestSet', 'F1 Score TestSet', 'Accuracy TestSet'])
df2

,Binary Relavance without Undersampling,Binary Relavance with Undersampling,Optimal Binary Relavance without Undersampling (F1 score),Optimal Binary Relavance with Undersampling (F1 score),Classifier Chain (CC),Optimal Classifier Chains (F1 score),Optimal Changed Chain Sequence CC
Hamming Loss ValidSet,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F1 Score ValidSet,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accuracy ValidSet,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hamming Loss TestSet,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F1 Score TestSet,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accuracy TestSet,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Task 1: Implement the Binary Relevance Algorithm

References: 
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
<br>
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn



In [7]:
class BinaryRel(BaseEstimator, ClassifierMixin):
    
    
    def __init__(self, model = 'DecisionTree', balanced='unbalanced'):
        self.model = model
        self.balanced = balanced
        
    
    def undersample(self, X_train, y_train):
        # Random Undersampling
        X_train_class_0 = X_train[y_train == 0]
        X_train_class_1 = X_train[y_train == 1]
        count_class_0 = X_train_class_0.iloc[:,0].count() #count number of each class (0 and 1)
        count_class_1 = X_train_class_1.iloc[:,0].count()
        if count_class_0 > count_class_1: #if 0 class greater than 1 class, randomly sample the 0 class features from the dataframe
            x_train_0_under = X_train_class_0.sample(count_class_1)
            y_train_0_under = pd.DataFrame(np.zeros((count_class_1, 1)))
            x_train_under = pd.concat([x_train_0_under, X_train_class_1], axis=0)
            y_train_under = pd.concat([y_train_0_under, pd.DataFrame(np.ones((count_class_1, 1)))], axis=0)
        else:
            x_train_1_under = X_train_class_1.sample(count_class_0)
            y_train_1_under = pd.DataFrame(np.ones((count_class_0, 1)))
            x_train_under = pd.concat([X_train_class_0, x_train_1_under], axis=0)
            y_train_under = pd.concat([pd.DataFrame(np.zeros((count_class_0, 1))),y_train_1_under], axis=0)
        
        return x_train_under,y_train_under
    
    def fit(self, features, labels):
        #fit function fits a model per label based on the base classifier specified and keeps all the models in a list that is iteratively appended
        
        if self.model == 'DecisionTree':
            our_model = tree.DecisionTreeClassifier(criterion="entropy")
        elif self.model == 'kNN':
            our_model = neighbors.KNeighborsClassifier()
        elif self.model == 'SVC':
            our_model = SVC(kernel='rbf')
        elif self.model == 'NaiveBayes':
            our_model = GaussianNB()
        elif self.model == 'LogisticRegression':
            our_model = linear_model.LogisticRegression()
        elif self.model == 'Bagging':
            our_model = ensemble.BaggingClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 50), n_estimators=10)
        elif self.model == 'Boosting':
            our_model = ensemble.AdaBoostClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 50), n_estimators=10)
        elif self.model == 'RandomForest':
            our_model = ensemble.RandomForestClassifier(n_estimators = 200, max_features=15, min_samples_split=100)

        our_final_model = list()
        
        if self.balanced == 'balanced': #check if user requests for balanced data with undersampling
            for i in range(len(labels.columns)):
                features_under, labels_under = self.undersample(features,labels.iloc[:,i])
                our_model = our_model.fit(features_under, labels_under.iloc[:,0])
                self.length_ = len(labels.columns)
                our_final_model.append(copy.deepcopy(our_model))
        else:
            for i in range(len(labels.columns)):
                our_model = our_model.fit(features, labels.iloc[:,i])
                self.length_ = len(labels.columns)
                our_final_model.append(copy.deepcopy(our_model))
      
        self.our_final_model_ = our_final_model
 
        return self

    def predict_proba(self, features_test):
        prediction_proba=np.zeros((len(features_test),2,self.length_))
        for k in range(self.length_):
            each_model=self.our_final_model_[k]
            prediction_proba[:,:,k]=each_model.predict_proba(features_test)
            
        return prediction_proba
            
    def predict(self, features_test):
        #uses the models found during fit and returns predictions as a matrix
        prediction = np.ones((len(features_test), self.length_))
        for k in range(self.length_):
            each_model = self.our_final_model_[k]
            prediction[:,k] = (each_model.predict(features_test)).T

        return prediction

In [8]:
X_train_plus_valid, X_test, y_train_plus_valid, y_test = train_test_split(features, labels, random_state=0, train_size = 0.7)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_plus_valid, y_train_plus_valid, random_state=0, train_size = 0.5/0.7)

In [9]:
our_model = BinaryRel(model = 'DecisionTree')
our_model.fit(X_train, y_train)

BinaryRel(balanced='unbalanced', model='DecisionTree')

In [10]:
prediction_proba_valid_nosamp=our_model.predict_proba(X_valid)
prediction_proba_valid_nosamp

array([[[1., 1., 1., ..., 1., 0., 1.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[1., 0., 0., ..., 0., 0., 1.],
        [0., 1., 1., ..., 1., 1., 0.]],

       [[1., 1., 1., ..., 0., 0., 1.],
        [0., 0., 0., ..., 1., 1., 0.]],

       ...,

       [[1., 1., 0., ..., 1., 0., 1.],
        [0., 0., 1., ..., 0., 1., 0.]],

       [[1., 1., 0., ..., 0., 0., 1.],
        [0., 0., 1., ..., 1., 1., 0.]],

       [[1., 1., 0., ..., 0., 0., 1.],
        [0., 0., 1., ..., 1., 1., 0.]]])

To show performance of predict_proba, we show with a naibe bayes classsifier, so that we can see probabilities.

In [84]:
our_modelnb = BinaryRel(model = 'NaiveBayes')
our_modelnb.fit(X_train, y_train)

BinaryRel(balanced='unbalanced', model='NaiveBayes')

In [85]:
prediction_proba_valid_nosamp_nb=our_modelnb.predict_proba(X_valid)
prediction_proba_valid_nosamp_nb

array([[[7.30968026e-05, 1.24090631e-01, 9.99529574e-01, ...,
         4.83457033e-01, 7.70364660e-01, 1.00000000e+00],
        [9.99926903e-01, 8.75909369e-01, 4.70425545e-04, ...,
         5.16542967e-01, 2.29635340e-01, 4.57032287e-12]],

       [[8.55812873e-01, 3.34898906e-01, 6.27773410e-02, ...,
         8.78547885e-01, 8.98386463e-01, 9.77328737e-01],
        [1.44187127e-01, 6.65101094e-01, 9.37222659e-01, ...,
         1.21452115e-01, 1.01613537e-01, 2.26712631e-02]],

       [[9.99847072e-01, 9.99758120e-01, 9.99730909e-01, ...,
         1.85365381e-05, 2.82443592e-05, 9.27370310e-01],
        [1.52928272e-04, 2.41879581e-04, 2.69090842e-04, ...,
         9.99981463e-01, 9.99971756e-01, 7.26296901e-02]],

       ...,

       [[9.99999862e-01, 9.99999073e-01, 6.53025475e-01, ...,
         2.53454299e-04, 1.96871547e-04, 9.19902229e-01],
        [1.37647663e-07, 9.26633563e-07, 3.46974525e-01, ...,
         9.99746546e-01, 9.99803128e-01, 8.00977715e-02]],

       [[3.77959744

In [11]:
# Predicting with validation set
prediction_valid_nosamp = our_model.predict(X_valid)

In [12]:
#to sort classes based on best accuracy for each class for classifier chains
accuracy=np.zeros((14,1))
for i in range(14):
    accuracy[i]=accuracy_score(y_valid.iloc[:,i], prediction_valid_nosamp[:,i])
new_column=np.argsort(accuracy,0)  
new_column=new_column[::-1]

The evaluation metrics commonly used for a multi-label classification problem are hamming loss and macro-averaged F1-score.

In [13]:
# Performance metric calculation (hamming loss and f1_score)
hammingloss_valid_nosamp = hamming_loss(y_valid, prediction_valid_nosamp)
print("Hamming loss: " + str(hammingloss_valid_nosamp))
f1_score_valid_nosamp = metrics.f1_score(y_valid, prediction_valid_nosamp,average='macro')
print("Macro-averaged F1 score: " + str(f1_score_valid_nosamp))

Hamming loss: 0.2669716646989374
Macro-averaged F1 score: 0.40790487583203255


In [14]:
#Accuracy calculation (for comparison)
accuracy_valid_nosamp = accuracy_score(y_valid, prediction_valid_nosamp)
print("Accuracy: " +  str(accuracy_valid_nosamp))
print(metrics.classification_report(y_valid, prediction_valid_nosamp))

Accuracy: 0.0371900826446281
              precision    recall  f1-score   support

           0       0.45      0.45      0.45       147
           1       0.52      0.51      0.51       207
           2       0.57      0.54      0.55       204
           3       0.59      0.56      0.58       193
           4       0.51      0.49      0.50       146
           5       0.35      0.41      0.38       111
           6       0.32      0.36      0.34        84
           7       0.36      0.38      0.37        97
           8       0.09      0.08      0.09        36
           9       0.15      0.19      0.17        53
          10       0.22      0.18      0.20        61
          11       0.80      0.81      0.80       368
          12       0.76      0.78      0.77       364
          13       0.00      0.00      0.00         6

   micro avg       0.56      0.57      0.57      2077
   macro avg       0.41      0.41      0.41      2077
weighted avg       0.57      0.57      0.57      20

C:\Users\Arlene John\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
# Check performance with pre-built algorithm from scikit-multilearn
# Reference: http://scikit.ml/tutorial.html

check_alg = BinaryRelevance(classifier = tree.DecisionTreeClassifier(), require_dense=None)
tr = check_alg.fit(X_train, y_train)
pred = check_alg.predict(X_test)
pred_ham = hamming_loss(y_test, pred)
pred_f1 = metrics.f1_score(y_test, pred,average='macro')
acc_check = accuracy_score(y_test, pred)
print("Hamming loss: " + str(pred_ham))
print("Macro-averaged F1 score: " + str(pred_f1))
print("Accuracy: " + str(acc_check))

Hamming loss: 0.28000787091696183
Macro-averaged F1 score: 0.3942115952516226
Accuracy: 0.03168044077134986


If we compare the performance of our algorithm (below) and from a pre-built binary classifier (above) from the skmultilearn library applied to the test set, it is possible to observe that the hamming loss and F1 scores are very similar, almost identical, to that of the classifier designed here. Therefore, our Binary Relevance algorithm can be validated and it is working as it should.

In [16]:
# Predicting with test set and evaluation metrics
prediction_test_nosamp = our_model.predict(X_test)
hammingloss_test_nosamp = hamming_loss(y_test, prediction_test_nosamp)
print("Hamming loss: " + str(hammingloss_test_nosamp))
f1_score_test_nosamp = metrics.f1_score(y_test, prediction_test_nosamp,average='macro')
print("Macro-averaged F1 score: " + str(f1_score_test_nosamp))
multilabel_confusion_matrix(y_test, prediction_test_nosamp)

Hamming loss: 0.2730224321133412
Macro-averaged F1 score: 0.3981049811365608


array([[[385, 103],
        [107, 131]],

       [[240, 165],
        [152, 169]],

       [[296, 147],
        [132, 151]],

       [[357, 125],
        [117, 127]],

       [[395, 114],
        [116, 101]],

       [[411, 139],
        [105,  71]],

       [[507, 104],
        [ 80,  35]],

       [[507,  80],
        [ 91,  48]],

       [[625,  43],
        [ 49,   9]],

       [[582,  69],
        [ 64,  11]],

       [[584,  62],
        [ 66,  14]],

       [[ 57, 137],
        [126, 406]],

       [[ 53, 146],
        [115, 412]],

       [[705,  11],
        [ 10,   0]]], dtype=int64)

In [17]:
accuracy_test_nosamp = accuracy_score(y_test, prediction_test_nosamp)
print("Accuracy: " +  str(accuracy_test_nosamp))
print(metrics.classification_report(y_test, prediction_test_nosamp))

Accuracy: 0.04269972451790634
              precision    recall  f1-score   support

           0       0.56      0.55      0.56       238
           1       0.51      0.53      0.52       321
           2       0.51      0.53      0.52       283
           3       0.50      0.52      0.51       244
           4       0.47      0.47      0.47       217
           5       0.34      0.40      0.37       176
           6       0.25      0.30      0.28       115
           7       0.38      0.35      0.36       139
           8       0.17      0.16      0.16        58
           9       0.14      0.15      0.14        75
          10       0.18      0.17      0.18        80
          11       0.75      0.76      0.76       532
          12       0.74      0.78      0.76       527
          13       0.00      0.00      0.00        10

   micro avg       0.54      0.56      0.55      3015
   macro avg       0.39      0.41      0.40      3015
weighted avg       0.54      0.56      0.55      3

In [18]:
# Store evaluation metrics in the dataframe
df['Binary Relavance (BR) without Undersampling']['Hamming Loss ValidSet'] = hammingloss_valid_nosamp
df['Binary Relavance (BR) without Undersampling']['F1 Score ValidSet'] = f1_score_valid_nosamp
df['Binary Relavance (BR) without Undersampling']['Accuracy ValidSet'] = accuracy_valid_nosamp
df['Binary Relavance (BR) without Undersampling']['Hamming Loss TestSet'] = hammingloss_test_nosamp
df['Binary Relavance (BR) without Undersampling']['F1 Score TestSet'] = f1_score_test_nosamp
df['Binary Relavance (BR) without Undersampling']['Accuracy TestSet'] = accuracy_test_nosamp

df2['Binary Relavance without Undersampling']['Hamming Loss ValidSet'] = hammingloss_valid_nosamp
df2['Binary Relavance without Undersampling']['F1 Score ValidSet'] = f1_score_valid_nosamp
df2['Binary Relavance without Undersampling']['Accuracy ValidSet'] = accuracy_valid_nosamp
df2['Binary Relavance without Undersampling']['Hamming Loss TestSet'] = hammingloss_test_nosamp
df2['Binary Relavance without Undersampling']['F1 Score TestSet'] = f1_score_test_nosamp
df2['Binary Relavance without Undersampling']['Accuracy TestSet'] = accuracy_test_nosamp

In [19]:
df

,Binary Relavance (BR) without Undersampling,Binary Relavance with Undersampling,Optimised BR Model with F1 score (w/o undersampling),Optimised BR Model with Accuracy (w/o undersampling),Optimised BR Model with F1 score (undersampled),Optimised BR Model with accuracy (undersampled)
Hamming Loss ValidSet,0.266972,0.0,0.0,0.0,0.0,0.0
F1 Score ValidSet,0.407905,0.0,0.0,0.0,0.0,0.0
Accuracy ValidSet,0.037190,0.0,0.0,0.0,0.0,0.0
Hamming Loss TestSet,0.273022,0.0,0.0,0.0,0.0,0.0
F1 Score TestSet,0.398105,0.0,0.0,0.0,0.0,0.0
Accuracy TestSet,0.042700,0.0,0.0,0.0,0.0,0.0


## Task 2: Implement the Binary Relevance Algorithm with Under-Sampling

In [20]:
our_model2 = BinaryRel(model = 'DecisionTree', balanced='balanced')
our_model2.fit(X_train, y_train)

BinaryRel(balanced='balanced', model='DecisionTree')

In [21]:
# Predicting with validation set and evaluation metrics
prediction_valid_samp = our_model2.predict(X_valid)
hammingloss_valid_samp = hamming_loss(y_valid, prediction_valid_samp)
f1_score_valid_samp = metrics.f1_score(y_valid, prediction_valid_samp,average='macro')
accuracy_valid_samp = accuracy_score(y_valid, prediction_valid_samp)
print("Hamming loss: " + str(hammingloss_valid_samp))
print("Macro-averaged F1 score: " + str(f1_score_valid_samp))
print("Accuracy: " +  str(accuracy_valid_samp))

Hamming loss: 0.423258559622196
Macro-averaged F1 score: 0.40745997794431066
Accuracy: 0.0


In [22]:
print(metrics.classification_report(y_valid, prediction_valid_samp))

              precision    recall  f1-score   support

           0       0.42      0.62      0.50       147
           1       0.52      0.56      0.54       207
           2       0.58      0.66      0.61       204
           3       0.51      0.59      0.55       193
           4       0.45      0.66      0.54       146
           5       0.31      0.61      0.41       111
           6       0.23      0.63      0.34        84
           7       0.24      0.56      0.34        97
           8       0.09      0.58      0.16        36
           9       0.12      0.49      0.19        53
          10       0.15      0.57      0.24        61
          11       0.78      0.49      0.61       368
          12       0.79      0.57      0.66       364
          13       0.01      0.50      0.02         6

   micro avg       0.38      0.58      0.46      2077
   macro avg       0.37      0.58      0.41      2077
weighted avg       0.54      0.58      0.53      2077
 samples avg       0.38   

In [23]:
# Predicting with test set and evaluation metrics
prediction_test_samp = our_model2.predict(X_test)
hammingloss_test_samp = hamming_loss(y_test, prediction_test_samp)
f1_score_test_samp = metrics.f1_score(y_test, prediction_test_samp,average='macro')
accuracy_test_samp = accuracy_score(y_test, prediction_test_samp)
print("Hamming loss: " + str(hammingloss_test_samp))
print("Macro-averaged F1 score: " + str(f1_score_test_samp))
print("Accuracy: " +  str(accuracy_test_samp))
multilabel_confusion_matrix(y_test, prediction_test_samp)

Hamming loss: 0.42384887839433294
Macro-averaged F1 score: 0.39557056936277074
Accuracy: 0.0027548209366391185


array([[[316, 172],
        [ 85, 153]],

       [[221, 184],
        [133, 188]],

       [[273, 170],
        [113, 170]],

       [[293, 189],
        [ 93, 151]],

       [[322, 187],
        [ 75, 142]],

       [[333, 217],
        [ 82,  94]],

       [[339, 272],
        [ 51,  64]],

       [[349, 238],
        [ 44,  95]],

       [[379, 289],
        [ 27,  31]],

       [[400, 251],
        [ 44,  31]],

       [[352, 294],
        [ 39,  41]],

       [[105,  89],
        [276, 256]],

       [[110,  89],
        [233, 294]],

       [[348, 368],
        [  4,   6]]], dtype=int64)

In [24]:
print("Accuracy: " +  str(accuracy_test_samp))
print(metrics.classification_report(y_test, prediction_test_samp))

Accuracy: 0.0027548209366391185
              precision    recall  f1-score   support

           0       0.47      0.64      0.54       238
           1       0.51      0.59      0.54       321
           2       0.50      0.60      0.55       283
           3       0.44      0.62      0.52       244
           4       0.43      0.65      0.52       217
           5       0.30      0.53      0.39       176
           6       0.19      0.56      0.28       115
           7       0.29      0.68      0.40       139
           8       0.10      0.53      0.16        58
           9       0.11      0.41      0.17        75
          10       0.12      0.51      0.20        80
          11       0.74      0.48      0.58       532
          12       0.77      0.56      0.65       527
          13       0.02      0.60      0.03        10

   micro avg       0.36      0.57      0.44      3015
   macro avg       0.36      0.57      0.40      3015
weighted avg       0.52      0.57      0.51     

In [25]:
#store evaluation metrics in dataframe
df['Binary Relavance with Undersampling']['Hamming Loss ValidSet'] = hammingloss_valid_samp
df['Binary Relavance with Undersampling']['F1 Score ValidSet'] = f1_score_valid_samp
df['Binary Relavance with Undersampling']['Accuracy ValidSet'] = accuracy_valid_samp
df['Binary Relavance with Undersampling']['Hamming Loss TestSet'] = hammingloss_test_samp
df['Binary Relavance with Undersampling']['F1 Score TestSet'] = f1_score_test_samp
df['Binary Relavance with Undersampling']['Accuracy TestSet'] = accuracy_test_samp

df2['Binary Relavance with Undersampling']['Hamming Loss ValidSet'] = hammingloss_valid_samp
df2['Binary Relavance with Undersampling']['F1 Score ValidSet'] = f1_score_valid_samp
df2['Binary Relavance with Undersampling']['Accuracy ValidSet'] = accuracy_valid_samp
df2['Binary Relavance with Undersampling']['Hamming Loss TestSet'] = hammingloss_test_samp
df2['Binary Relavance with Undersampling']['F1 Score TestSet'] = f1_score_test_samp
df2['Binary Relavance with Undersampling']['Accuracy TestSet'] = accuracy_test_samp

In [26]:
df

,Binary Relavance (BR) without Undersampling,Binary Relavance with Undersampling,Optimised BR Model with F1 score (w/o undersampling),Optimised BR Model with Accuracy (w/o undersampling),Optimised BR Model with F1 score (undersampled),Optimised BR Model with accuracy (undersampled)
Hamming Loss ValidSet,0.266972,0.423259,0.0,0.0,0.0,0.0
F1 Score ValidSet,0.407905,0.407460,0.0,0.0,0.0,0.0
Accuracy ValidSet,0.037190,0.000000,0.0,0.0,0.0,0.0
Hamming Loss TestSet,0.273022,0.423849,0.0,0.0,0.0,0.0
F1 Score TestSet,0.398105,0.395571,0.0,0.0,0.0,0.0
Accuracy TestSet,0.042700,0.002755,0.0,0.0,0.0,0.0


It is possible to observe an almost 10 % increase in hamming loss for both the validation set and the test set when undersampling is applied, showing performance drop when undersampling is applied. However, evaluating the model with the F1-score, which is a good measure when we have imbalanced classes, there is no improvement. In reality, the model without under-sampling has a better score when we compare the F1-scores, hamming loss, and accuracy. F1-score does not improve because the F1 score is already a measure that accounts for imbalanced classes. Since we are attempting to correct that with undersampling, no big changes in the F1 score can be observed. Besides, undersampling loses information and that information can be important and useful. With that being said, the undersampling does not improve the performance of the Binary Relevance classifier greatly, however from the confusion matrices, it can be observed that the matrix after the undersampling has more balanced results. </br>

It is important to observe, though, that these results are with the Decision Tree classifier as the base classifier. If the base classifier is changed, the metrics may improve with undersampling. Therefore, a good way to evaluate all the possibilities would be to run a Grid Search with all the base classifiers and with and without undersampling.


## Task 3: Compare the Performance of Different Binary Relevance Approaches

### Find best models without undersampling

In [27]:
param_grid = [{'model':('DecisionTree','kNN','SVC','NaiveBayes','LogisticRegression', 'Bagging', 'Boosting', 'RandomForest')}]

In [28]:
#grid search for best base classifier optimized for f1-score
best_model = GridSearchCV(BinaryRel(balanced='unbalanced'),param_grid=param_grid,cv=10,n_jobs=-1, scoring='f1_macro')
best_model.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=BinaryRel(balanced='unbalanced', model='DecisionTree'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'model': ('DecisionTree', 'kNN', 'SVC', 'NaiveBayes',
                                    'LogisticRegression', 'Bagging', 'Boosting',
                                    'RandomForest')}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [29]:
print("Best parameters set found on development set:")
print(best_model.best_params_)
print(best_model.best_score_)

Best parameters set found on development set:
{'model': 'NaiveBayes'}
0.4416458137427218


In [30]:
#prediction on validation set and evaluation metrics
prediction_valid_opt1 = best_model.predict(X_valid)
hammingloss_valid_opt1 = hamming_loss(y_valid,prediction_valid_opt1)
f1_score_valid_opt1 = metrics.f1_score(y_valid,prediction_valid_opt1,average='macro')
accuracy_valid_opt1 = accuracy_score(y_valid, prediction_valid_opt1)
print("Hamming loss: " + str(hammingloss_valid_opt1))
print("Macro-averaged F1 score: " + str(f1_score_valid_opt1))
print("Accuracy: " +  str(accuracy_valid_opt1))
multilabel_confusion_matrix(y_valid,prediction_valid_opt1)

Hamming loss: 0.28733766233766234
Macro-averaged F1 score: 0.4814404642129134
Accuracy: 0.10330578512396695


array([[[265,  72],
        [ 58,  89]],

       [[155, 122],
        [ 61, 146]],

       [[196,  84],
        [ 52, 152]],

       [[237,  54],
        [ 81, 112]],

       [[272,  66],
        [ 68,  78]],

       [[267, 106],
        [ 44,  67]],

       [[277, 123],
        [ 40,  44]],

       [[273, 114],
        [ 41,  56]],

       [[342, 106],
        [ 20,  16]],

       [[326, 105],
        [ 28,  25]],

       [[328,  95],
        [ 35,  26]],

       [[ 72,  44],
        [128, 240]],

       [[ 76,  44],
        [128, 236]],

       [[452,  26],
        [  2,   4]]], dtype=int64)

In [31]:
print("Accuracy: " +  str(accuracy_valid_opt1))
print(metrics.classification_report(y_valid, prediction_valid_opt1))

Accuracy: 0.10330578512396695
              precision    recall  f1-score   support

           0       0.55      0.61      0.58       147
           1       0.54      0.71      0.61       207
           2       0.64      0.75      0.69       204
           3       0.67      0.58      0.62       193
           4       0.54      0.53      0.54       146
           5       0.39      0.60      0.47       111
           6       0.26      0.52      0.35        84
           7       0.33      0.58      0.42        97
           8       0.13      0.44      0.20        36
           9       0.19      0.47      0.27        53
          10       0.21      0.43      0.29        61
          11       0.85      0.65      0.74       368
          12       0.84      0.65      0.73       364
          13       0.13      0.67      0.22         6

   micro avg       0.53      0.62      0.57      2077
   macro avg       0.45      0.58      0.48      2077
weighted avg       0.62      0.62      0.60      2

In [32]:
#prediction on test set and evaluation metrics
prediction_test_opt1 = best_model.predict(X_test)
hammingloss_test_opt1 = hamming_loss(y_test,prediction_test_opt1)
f1_score_test_opt1 = metrics.f1_score(y_test,prediction_test_opt1,average='macro')
accuracy_test_opt1 = accuracy_score(y_test, prediction_test_opt1)
print("Hamming loss: " + str(hammingloss_test_opt1))
print("Macro-averaged F1 score: " + str(f1_score_test_opt1))
print("Accuracy: " +  str(accuracy_test_opt1))
multilabel_confusion_matrix(y_test,prediction_test_opt1)

Hamming loss: 0.29338842975206614
Macro-averaged F1 score: 0.44515024684090815
Accuracy: 0.09779614325068871


array([[[392,  96],
        [ 80, 158]],

       [[231, 174],
        [121, 200]],

       [[295, 148],
        [ 69, 214]],

       [[351, 131],
        [103, 141]],

       [[405, 104],
        [100, 117]],

       [[397, 153],
        [ 81,  95]],

       [[450, 161],
        [ 51,  64]],

       [[426, 161],
        [ 72,  67]],

       [[535, 133],
        [ 43,  15]],

       [[519, 132],
        [ 42,  33]],

       [[495, 151],
        [ 54,  26]],

       [[ 92, 102],
        [181, 351]],

       [[ 94, 105],
        [180, 347]],

       [[671,  45],
        [  9,   1]]], dtype=int64)

In [33]:
print("Accuracy: " +  str(accuracy_test_opt1))
print(metrics.classification_report(y_test, prediction_test_opt1))

Accuracy: 0.09779614325068871
              precision    recall  f1-score   support

           0       0.62      0.66      0.64       238
           1       0.53      0.62      0.58       321
           2       0.59      0.76      0.66       283
           3       0.52      0.58      0.55       244
           4       0.53      0.54      0.53       217
           5       0.38      0.54      0.45       176
           6       0.28      0.56      0.38       115
           7       0.29      0.48      0.37       139
           8       0.10      0.26      0.15        58
           9       0.20      0.44      0.28        75
          10       0.15      0.33      0.20        80
          11       0.77      0.66      0.71       532
          12       0.77      0.66      0.71       527
          13       0.02      0.10      0.04        10

   micro avg       0.50      0.61      0.55      3015
   macro avg       0.41      0.51      0.45      3015
weighted avg       0.57      0.61      0.58      3

In [34]:
#store evaluation metrics in the dataframe
df['Optimised BR Model with F1 score (w/o undersampling)']['Hamming Loss ValidSet'] = hammingloss_valid_opt1
df['Optimised BR Model with F1 score (w/o undersampling)']['F1 Score ValidSet'] = f1_score_valid_opt1
df['Optimised BR Model with F1 score (w/o undersampling)']['Accuracy ValidSet'] = accuracy_valid_opt1
df['Optimised BR Model with F1 score (w/o undersampling)']['Hamming Loss TestSet'] = hammingloss_test_opt1
df['Optimised BR Model with F1 score (w/o undersampling)']['F1 Score TestSet'] = f1_score_test_opt1
df['Optimised BR Model with F1 score (w/o undersampling)']['Accuracy TestSet'] = accuracy_test_opt1

df2['Optimal Binary Relavance without Undersampling (F1 score)']['Hamming Loss ValidSet'] = hammingloss_valid_opt1
df2['Optimal Binary Relavance without Undersampling (F1 score)']['F1 Score ValidSet'] = f1_score_valid_opt1
df2['Optimal Binary Relavance without Undersampling (F1 score)']['Accuracy ValidSet'] = accuracy_valid_opt1
df2['Optimal Binary Relavance without Undersampling (F1 score)']['Hamming Loss TestSet'] = hammingloss_test_opt1
df2['Optimal Binary Relavance without Undersampling (F1 score)']['F1 Score TestSet'] = f1_score_test_opt1
df2['Optimal Binary Relavance without Undersampling (F1 score)']['Accuracy TestSet'] = accuracy_test_opt1

In [35]:
df

,Binary Relavance (BR) without Undersampling,Binary Relavance with Undersampling,Optimised BR Model with F1 score (w/o undersampling),Optimised BR Model with Accuracy (w/o undersampling),Optimised BR Model with F1 score (undersampled),Optimised BR Model with accuracy (undersampled)
Hamming Loss ValidSet,0.266972,0.423259,0.287338,0.0,0.0,0.0
F1 Score ValidSet,0.407905,0.407460,0.481440,0.0,0.0,0.0
Accuracy ValidSet,0.037190,0.000000,0.103306,0.0,0.0,0.0
Hamming Loss TestSet,0.273022,0.423849,0.293388,0.0,0.0,0.0
F1 Score TestSet,0.398105,0.395571,0.445150,0.0,0.0,0.0
Accuracy TestSet,0.042700,0.002755,0.097796,0.0,0.0,0.0


Optimise the model for accuracy

In [36]:
#grid search for best base classifier optimized for accuracy
best_model = GridSearchCV(BinaryRel(balanced='unbalanced'),param_grid=param_grid,cv=10,n_jobs=-1, scoring='accuracy')
best_model.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=BinaryRel(balanced='unbalanced', model='DecisionTree'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'model': ('DecisionTree', 'kNN', 'SVC', 'NaiveBayes',
                                    'LogisticRegression', 'Bagging', 'Boosting',
                                    'RandomForest')}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [37]:
print("Best parameters set found on development set:")
print(best_model.best_params_)
print(best_model.best_score_)

Best parameters set found on development set:
{'model': 'kNN'}
0.19553719008264464


In [38]:
#prediction on validation set and evaluation metrics
prediction_valid_opt2 = best_model.predict(X_valid)
hammingloss_valid_opt2 = hamming_loss(y_valid,prediction_valid_opt2)
f1_score_valid_opt2 = metrics.f1_score(y_valid,prediction_valid_opt2,average='macro')
accuracy_valid_opt2 = accuracy_score(y_valid, prediction_valid_opt2)
print("Hamming loss: " + str(hammingloss_valid_opt2))
print("Macro-averaged F1 score: " + str(f1_score_valid_opt2))
print("Accuracy: " +  str(accuracy_valid_opt2))
multilabel_confusion_matrix(y_valid,prediction_valid_opt2)

Hamming loss: 0.2051357733175915
Macro-averaged F1 score: 0.439516029675847
Accuracy: 0.2066115702479339


array([[[304,  33],
        [ 81,  66]],

       [[199,  78],
        [ 87, 120]],

       [[214,  66],
        [ 73, 131]],

       [[243,  48],
        [ 82, 111]],

       [[284,  54],
        [ 58,  88]],

       [[332,  41],
        [ 70,  41]],

       [[379,  21],
        [ 65,  19]],

       [[360,  27],
        [ 79,  18]],

       [[447,   1],
        [ 35,   1]],

       [[420,  11],
        [ 45,   8]],

       [[413,  10],
        [ 53,   8]],

       [[ 18,  98],
        [ 34, 334]],

       [[ 19, 101],
        [ 33, 331]],

       [[478,   0],
        [  6,   0]]], dtype=int64)

In [39]:
print("Accuracy: " +  str(accuracy_valid_opt2))
print(metrics.classification_report(y_valid, prediction_valid_opt2))

Accuracy: 0.2066115702479339
              precision    recall  f1-score   support

           0       0.67      0.45      0.54       147
           1       0.61      0.58      0.59       207
           2       0.66      0.64      0.65       204
           3       0.70      0.58      0.63       193
           4       0.62      0.60      0.61       146
           5       0.50      0.37      0.42       111
           6       0.47      0.23      0.31        84
           7       0.40      0.19      0.25        97
           8       0.50      0.03      0.05        36
           9       0.42      0.15      0.22        53
          10       0.44      0.13      0.20        61
          11       0.77      0.91      0.83       368
          12       0.77      0.91      0.83       364
          13       0.00      0.00      0.00         6

   micro avg       0.68      0.61      0.65      2077
   macro avg       0.54      0.41      0.44      2077
weighted avg       0.65      0.61      0.62      20

C:\Users\Arlene John\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arlene John\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
#prediction on test set and evaluation metrics
prediction_test_opt2 = best_model.predict(X_test)
hammingloss_test_opt2 = hamming_loss(y_test, prediction_test_opt2)
f1_score_test_opt2 = metrics.f1_score(y_test, prediction_test_opt2,average='macro')
accuracy_test_opt2 = accuracy_score(y_test, prediction_test_opt2)
print("Hamming loss: " + str(hammingloss_test_opt2))
print("Macro-averaged F1 score: " + str(f1_score_test_opt2))
print("Accuracy: " +  str(accuracy_test_opt2))
multilabel_confusion_matrix(y_test,prediction_test_opt2)

Hamming loss: 0.2109405745769382
Macro-averaged F1 score: 0.40854376279464155
Accuracy: 0.1790633608815427


array([[[439,  49],
        [129, 109]],

       [[284, 121],
        [152, 169]],

       [[318, 125],
        [106, 177]],

       [[390,  92],
        [105, 139]],

       [[444,  65],
        [ 98, 119]],

       [[512,  38],
        [118,  58]],

       [[580,  31],
        [ 90,  25]],

       [[552,  35],
        [113,  26]],

       [[666,   2],
        [ 57,   1]],

       [[638,  13],
        [ 70,   5]],

       [[634,  12],
        [ 76,   4]],

       [[ 26, 168],
        [ 49, 483]],

       [[ 28, 171],
        [ 49, 478]],

       [[716,   0],
        [ 10,   0]]], dtype=int64)

In [41]:
print("Accuracy: " +  str(accuracy_test_opt2))
print(metrics.classification_report(y_test, prediction_test_opt2))

Accuracy: 0.1790633608815427
              precision    recall  f1-score   support

           0       0.69      0.46      0.55       238
           1       0.58      0.53      0.55       321
           2       0.59      0.63      0.61       283
           3       0.60      0.57      0.59       244
           4       0.65      0.55      0.59       217
           5       0.60      0.33      0.43       176
           6       0.45      0.22      0.29       115
           7       0.43      0.19      0.26       139
           8       0.33      0.02      0.03        58
           9       0.28      0.07      0.11        75
          10       0.25      0.05      0.08        80
          11       0.74      0.91      0.82       532
          12       0.74      0.91      0.81       527
          13       0.00      0.00      0.00        10

   micro avg       0.66      0.59      0.63      3015
   macro avg       0.49      0.39      0.41      3015
weighted avg       0.62      0.59      0.59      30

C:\Users\Arlene John\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Arlene John\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
#store evaluation metrics in the dataframe
df['Optimised BR Model with Accuracy (w/o undersampling)']['Hamming Loss ValidSet'] = hammingloss_valid_opt2
df['Optimised BR Model with Accuracy (w/o undersampling)']['F1 Score ValidSet'] = f1_score_valid_opt2
df['Optimised BR Model with Accuracy (w/o undersampling)']['Accuracy ValidSet'] = accuracy_valid_opt2
df['Optimised BR Model with Accuracy (w/o undersampling)']['Hamming Loss TestSet'] = hammingloss_test_opt2
df['Optimised BR Model with Accuracy (w/o undersampling)']['F1 Score TestSet'] = f1_score_test_opt2
df['Optimised BR Model with Accuracy (w/o undersampling)']['Accuracy TestSet'] = accuracy_test_opt2

In [43]:
df

,Binary Relavance (BR) without Undersampling,Binary Relavance with Undersampling,Optimised BR Model with F1 score (w/o undersampling),Optimised BR Model with Accuracy (w/o undersampling),Optimised BR Model with F1 score (undersampled),Optimised BR Model with accuracy (undersampled)
Hamming Loss ValidSet,0.266972,0.423259,0.287338,0.205136,0.0,0.0
F1 Score ValidSet,0.407905,0.407460,0.481440,0.439516,0.0,0.0
Accuracy ValidSet,0.037190,0.000000,0.103306,0.206612,0.0,0.0
Hamming Loss TestSet,0.273022,0.423849,0.293388,0.210941,0.0,0.0
F1 Score TestSet,0.398105,0.395571,0.445150,0.408544,0.0,0.0
Accuracy TestSet,0.042700,0.002755,0.097796,0.179063,0.0,0.0


The best parameters found with Grid Search optimized for the F1 score have indeed improved the F1 score and accuracy for the validation set and the test set. Since the Grid Search was optimized for the F1 score, this is expected. The Hamming Loss increased with the new model. It could because the Naive Bayes classifier is the most appropriate model to optimize the F1 score but might not be optimal for Hamming Loss. 

The best parameters found with Grid Search optimized for accuracy improved the F1 score and accuracy for the validation set and the test set. Since the Grid Search was optimized for accuracy, this is expected. The Hamming Loss decreased with the new model. This shows that even though the F1 score did not improve as much as when the model was optimized for the F1 score, there has been an overall improvement in performance when optimized for accuracy.

### Find best models with undersampling

In [44]:
#grid search for best base classifier optimized for f1-score
best_model = GridSearchCV(BinaryRel(balanced='balanced'),param_grid=param_grid,cv=10,n_jobs=-1, scoring='f1_macro')
best_model.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=BinaryRel(balanced='balanced', model='DecisionTree'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'model': ('DecisionTree', 'kNN', 'SVC', 'NaiveBayes',
                                    'LogisticRegression', 'Bagging', 'Boosting',
                                    'RandomForest')}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [45]:
print("Best parameters set found on development set:")
print(best_model.best_params_)
print(best_model.best_score_)

Best parameters set found on development set:
{'model': 'SVC'}
0.4560795077569534


In [46]:
#prediction on validation set and evaluation metrics
prediction_valid_opt3 = best_model.predict(X_valid)
hammingloss_valid_opt3 = hamming_loss(y_valid,prediction_valid_opt3)
f1_score_valid_opt3 = metrics.f1_score(y_valid,prediction_valid_opt3,average='macro')
accuracy_valid_opt3 = accuracy_score(y_valid, prediction_valid_opt3)
print("Hamming loss: " + str(hammingloss_valid_opt3))
print("Macro-averaged F1 score: " + str(f1_score_valid_opt3))
print("Accuracy: " +  str(accuracy_valid_opt3))
multilabel_confusion_matrix(y_valid,prediction_valid_opt3)

Hamming loss: 0.33840023612750886
Macro-averaged F1 score: 0.4801737032836975
Accuracy: 0.05991735537190083


array([[[259,  78],
        [ 46, 101]],

       [[173, 104],
        [ 57, 150]],

       [[206,  74],
        [ 52, 152]],

       [[221,  70],
        [ 42, 151]],

       [[281,  57],
        [ 50,  96]],

       [[251, 122],
        [ 42,  69]],

       [[297, 103],
        [ 40,  44]],

       [[219, 168],
        [ 35,  62]],

       [[232, 216],
        [ 10,  26]],

       [[266, 165],
        [ 16,  37]],

       [[244, 179],
        [ 24,  37]],

       [[ 83,  33],
        [153, 215]],

       [[ 88,  32],
        [155, 209]],

       [[312, 166],
        [  4,   2]]], dtype=int64)

In [47]:
print("Accuracy: " +  str(accuracy_valid_opt3))
print(metrics.classification_report(y_valid, prediction_valid_opt3))

Accuracy: 0.05991735537190083
              precision    recall  f1-score   support

           0       0.56      0.69      0.62       147
           1       0.59      0.72      0.65       207
           2       0.67      0.75      0.71       204
           3       0.68      0.78      0.73       193
           4       0.63      0.66      0.64       146
           5       0.36      0.62      0.46       111
           6       0.30      0.52      0.38        84
           7       0.27      0.64      0.38        97
           8       0.11      0.72      0.19        36
           9       0.18      0.70      0.29        53
          10       0.17      0.61      0.27        61
          11       0.87      0.58      0.70       368
          12       0.87      0.57      0.69       364
          13       0.01      0.33      0.02         6

   micro avg       0.46      0.65      0.54      2077
   macro avg       0.45      0.64      0.48      2077
weighted avg       0.63      0.65      0.61      2

In [48]:
#prediction on test set and evaluation metrics
prediction_test_opt3 = best_model.predict(X_test)
hammingloss_test_opt3 = hamming_loss(y_test, prediction_test_opt3)
f1_score_test_opt3 = metrics.f1_score(y_test, prediction_test_opt3,average='macro')
accuracy_test_opt3 = accuracy_score(y_test, prediction_test_opt3)
print("Hamming loss: " + str(hammingloss_test_opt3))
print("Macro-averaged F1 score: " + str(f1_score_test_opt3))
print("Accuracy: " +  str(accuracy_test_opt3))
multilabel_confusion_matrix(y_test,prediction_test_opt3)

Hamming loss: 0.3492719401810311
Macro-averaged F1 score: 0.45129654161761285
Accuracy: 0.07024793388429752


array([[[387, 101],
        [ 77, 161]],

       [[253, 152],
        [114, 207]],

       [[308, 135],
        [ 80, 203]],

       [[330, 152],
        [ 62, 182]],

       [[417,  92],
        [ 80, 137]],

       [[362, 188],
        [ 60, 116]],

       [[451, 160],
        [ 57,  58]],

       [[367, 220],
        [ 56,  83]],

       [[371, 297],
        [ 27,  31]],

       [[410, 241],
        [ 32,  43]],

       [[403, 243],
        [ 36,  44]],

       [[117,  77],
        [243, 289]],

       [[120,  79],
        [249, 278]],

       [[480, 236],
        [  4,   6]]], dtype=int64)

In [49]:
print("Accuracy: " +  str(accuracy_test_opt3))
print(metrics.classification_report(y_test, prediction_test_opt3))

Accuracy: 0.07024793388429752
              precision    recall  f1-score   support

           0       0.61      0.68      0.64       238
           1       0.58      0.64      0.61       321
           2       0.60      0.72      0.65       283
           3       0.54      0.75      0.63       244
           4       0.60      0.63      0.61       217
           5       0.38      0.66      0.48       176
           6       0.27      0.50      0.35       115
           7       0.27      0.60      0.38       139
           8       0.09      0.53      0.16        58
           9       0.15      0.57      0.24        75
          10       0.15      0.55      0.24        80
          11       0.79      0.54      0.64       532
          12       0.78      0.53      0.63       527
          13       0.02      0.60      0.05        10

   micro avg       0.44      0.61      0.51      3015
   macro avg       0.42      0.61      0.45      3015
weighted avg       0.58      0.61      0.57      3

In [50]:
#store evaluation metrics in the dataframe
df['Optimised BR Model with F1 score (undersampled)']['Hamming Loss ValidSet'] = hammingloss_valid_opt3
df['Optimised BR Model with F1 score (undersampled)']['F1 Score ValidSet'] = f1_score_valid_opt3
df['Optimised BR Model with F1 score (undersampled)']['Accuracy ValidSet'] = accuracy_valid_opt3
df['Optimised BR Model with F1 score (undersampled)']['Hamming Loss TestSet'] = hammingloss_test_opt3
df['Optimised BR Model with F1 score (undersampled)']['F1 Score TestSet'] = f1_score_test_opt3
df['Optimised BR Model with F1 score (undersampled)']['Accuracy TestSet'] = accuracy_test_opt3

df2['Optimal Binary Relavance with Undersampling (F1 score)']['Hamming Loss ValidSet'] = hammingloss_valid_opt3
df2['Optimal Binary Relavance with Undersampling (F1 score)']['F1 Score ValidSet'] = f1_score_valid_opt3
df2['Optimal Binary Relavance with Undersampling (F1 score)']['Accuracy ValidSet'] = accuracy_valid_opt3
df2['Optimal Binary Relavance with Undersampling (F1 score)']['Hamming Loss TestSet'] = hammingloss_test_opt3
df2['Optimal Binary Relavance with Undersampling (F1 score)']['F1 Score TestSet'] = f1_score_test_opt3
df2['Optimal Binary Relavance with Undersampling (F1 score)']['Accuracy TestSet'] = accuracy_test_opt3

In [51]:
df

,Binary Relavance (BR) without Undersampling,Binary Relavance with Undersampling,Optimised BR Model with F1 score (w/o undersampling),Optimised BR Model with Accuracy (w/o undersampling),Optimised BR Model with F1 score (undersampled),Optimised BR Model with accuracy (undersampled)
Hamming Loss ValidSet,0.266972,0.423259,0.287338,0.205136,0.338400,0.0
F1 Score ValidSet,0.407905,0.407460,0.481440,0.439516,0.480174,0.0
Accuracy ValidSet,0.037190,0.000000,0.103306,0.206612,0.059917,0.0
Hamming Loss TestSet,0.273022,0.423849,0.293388,0.210941,0.349272,0.0
F1 Score TestSet,0.398105,0.395571,0.445150,0.408544,0.451297,0.0
Accuracy TestSet,0.042700,0.002755,0.097796,0.179063,0.070248,0.0


Optimize the undersampled model for accuracy

In [52]:
#grid search for best base classifier optimized for accuracy
best_model = GridSearchCV(BinaryRel(balanced='balanced'),param_grid=param_grid,cv=10,n_jobs=-1, scoring='accuracy')
best_model.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=BinaryRel(balanced='balanced', model='DecisionTree'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'model': ('DecisionTree', 'kNN', 'SVC', 'NaiveBayes',
                                    'LogisticRegression', 'Bagging', 'Boosting',
                                    'RandomForest')}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [53]:
print("Best parameters set found on development set:")
print(best_model.best_params_)
print(best_model.best_score_)

Best parameters set found on development set:
{'model': 'kNN'}
0.04474517906336088


In [54]:
#prediction on validation set and evaluation metrics
prediction_valid_opt4 = best_model.predict(X_valid)
hammingloss_valid_opt4 = hamming_loss(y_valid,prediction_valid_opt4)
f1_score_valid_opt4 = metrics.f1_score(y_valid,prediction_valid_opt4,average='macro')
accuracy_valid_opt4 = accuracy_score(y_valid, prediction_valid_opt4)
print("Hamming loss: " + str(hammingloss_valid_opt4))
print("Macro-averaged F1 score: " + str(f1_score_valid_opt4))
print("Accuracy: " +  str(accuracy_valid_opt4))
multilabel_confusion_matrix(y_valid,prediction_valid_opt4)

Hamming loss: 0.3769185360094451
Macro-averaged F1 score: 0.45732759634601133
Accuracy: 0.0640495867768595


array([[[241,  96],
        [ 53,  94]],

       [[168, 109],
        [ 69, 138]],

       [[192,  88],
        [ 54, 150]],

       [[214,  77],
        [ 56, 137]],

       [[219, 119],
        [ 43, 103]],

       [[221, 152],
        [ 37,  74]],

       [[238, 162],
        [ 22,  62]],

       [[210, 177],
        [ 34,  63]],

       [[236, 212],
        [ 13,  23]],

       [[251, 180],
        [ 20,  33]],

       [[249, 174],
        [ 24,  37]],

       [[ 68,  48],
        [159, 209]],

       [[ 70,  50],
        [136, 228]],

       [[290, 188],
        [  2,   4]]], dtype=int64)

In [55]:
print("Accuracy: " +  str(accuracy_valid_opt4))
print(metrics.classification_report(y_valid, prediction_valid_opt4))

Accuracy: 0.0640495867768595
              precision    recall  f1-score   support

           0       0.49      0.64      0.56       147
           1       0.56      0.67      0.61       207
           2       0.63      0.74      0.68       204
           3       0.64      0.71      0.67       193
           4       0.46      0.71      0.56       146
           5       0.33      0.67      0.44       111
           6       0.28      0.74      0.40        84
           7       0.26      0.65      0.37        97
           8       0.10      0.64      0.17        36
           9       0.15      0.62      0.25        53
          10       0.18      0.61      0.27        61
          11       0.81      0.57      0.67       368
          12       0.82      0.63      0.71       364
          13       0.02      0.67      0.04         6

   micro avg       0.43      0.65      0.51      2077
   macro avg       0.41      0.66      0.46      2077
weighted avg       0.58      0.65      0.59      20

In [56]:
#prediction on test set and evaluation metrics
prediction_test_opt4 = best_model.predict(X_test)
hammingloss_test_opt4 = hamming_loss(y_test, prediction_test_opt4)
f1_score_test_opt4 = metrics.f1_score(y_test, prediction_test_opt4,average='macro')
accuracy_test_opt4 = accuracy_score(y_test, prediction_test_opt4)
print("Hamming loss: " + str(hammingloss_test_opt4))
print("Macro-averaged F1 score: " + str(f1_score_test_opt4))
print("Accuracy: " +  str(accuracy_test_opt4))
multilabel_confusion_matrix(y_test,prediction_test_opt4)

Hamming loss: 0.37711530893349077
Macro-averaged F1 score: 0.4419340316987174
Accuracy: 0.06611570247933884


array([[[331, 157],
        [ 87, 151]],

       [[250, 155],
        [127, 194]],

       [[291, 152],
        [ 77, 206]],

       [[318, 164],
        [ 74, 170]],

       [[328, 181],
        [ 54, 163]],

       [[344, 206],
        [ 65, 111]],

       [[398, 213],
        [ 45,  70]],

       [[338, 249],
        [ 56,  83]],

       [[366, 302],
        [ 19,  39]],

       [[405, 246],
        [ 34,  41]],

       [[397, 249],
        [ 34,  46]],

       [[100,  94],
        [217, 315]],

       [[ 99, 100],
        [196, 331]],

       [[437, 279],
        [  1,   9]]], dtype=int64)

In [57]:
print("Accuracy: " +  str(accuracy_test_opt4))
print(metrics.classification_report(y_test, prediction_test_opt4))

Accuracy: 0.06611570247933884
              precision    recall  f1-score   support

           0       0.49      0.63      0.55       238
           1       0.56      0.60      0.58       321
           2       0.58      0.73      0.64       283
           3       0.51      0.70      0.59       244
           4       0.47      0.75      0.58       217
           5       0.35      0.63      0.45       176
           6       0.25      0.61      0.35       115
           7       0.25      0.60      0.35       139
           8       0.11      0.67      0.20        58
           9       0.14      0.55      0.23        75
          10       0.16      0.57      0.25        80
          11       0.77      0.59      0.67       532
          12       0.77      0.63      0.69       527
          13       0.03      0.90      0.06        10

   micro avg       0.41      0.64      0.50      3015
   macro avg       0.39      0.65      0.44      3015
weighted avg       0.55      0.64      0.57      3

In [58]:
#store evaluation metrics in the dataframe
df['Optimised BR Model with accuracy (undersampled)']['Hamming Loss ValidSet'] = hammingloss_valid_opt4
df['Optimised BR Model with accuracy (undersampled)']['F1 Score ValidSet'] = f1_score_valid_opt4
df['Optimised BR Model with accuracy (undersampled)']['Accuracy ValidSet'] = accuracy_valid_opt4
df['Optimised BR Model with accuracy (undersampled)']['Hamming Loss TestSet'] = hammingloss_test_opt4
df['Optimised BR Model with accuracy (undersampled)']['F1 Score TestSet'] = f1_score_test_opt4
df['Optimised BR Model with accuracy (undersampled)']['Accuracy TestSet'] = accuracy_test_opt4

In [59]:
df

,Binary Relavance (BR) without Undersampling,Binary Relavance with Undersampling,Optimised BR Model with F1 score (w/o undersampling),Optimised BR Model with Accuracy (w/o undersampling),Optimised BR Model with F1 score (undersampled),Optimised BR Model with accuracy (undersampled)
Hamming Loss ValidSet,0.266972,0.423259,0.287338,0.205136,0.338400,0.376919
F1 Score ValidSet,0.407905,0.407460,0.481440,0.439516,0.480174,0.457328
Accuracy ValidSet,0.037190,0.000000,0.103306,0.206612,0.059917,0.064050
Hamming Loss TestSet,0.273022,0.423849,0.293388,0.210941,0.349272,0.377115
F1 Score TestSet,0.398105,0.395571,0.445150,0.408544,0.451297,0.441934
Accuracy TestSet,0.042700,0.002755,0.097796,0.179063,0.070248,0.066116


The best parameters found with Grid Search optimized for the F1 score and undersampling have indeed improved the F1 score and accuracy for the validation set and the test set. Since the Grid Search was optimized for the F1 score, this is expected. The Hamming Loss decreased with the new model. <br/>

The best parameters found with Grid Search for accuracy improved the F1 score and accuracy for the validation set and the test set. Since the Grid Search was optimized for accuracy, this is expected. The Hamming Loss decreased with the new model. There has been an overall improvement in performance when optimized with accuracy, however, the improvements are better when the model is optimized for F1 score.

Base Classifiers (Reminder!):

Binary Relavance with and without undersampling: Decision Tree
 <br />
Optimised Model with F1 Score for Binary relevance classifiers: SVC with undersampling, Naive Bayes without undersampling
 <br />
Optimised Model with Accuracy: kNN with undersampling, kNN without undersampling

Therefore, based on the comparisons made, the Optimal model found by Grid Search optimized for F1-score, Naive Bayes without undersampling and SVC with undersampling are good models to evaluate and test the dataset using Binary Relevance. Accuracy may not be always a reliable measure to evaluate models.

## Task 4: Implement the Classifier Chains Algorithm

In [60]:
class ClassifierChainImplement(BaseEstimator, ClassifierMixin):
    
    
    def __init__(self, model = 'DecisionTree'):
        self.model = model
    
   
    def fit(self, X_train, Y_train):
    #fit function fits a model per label based on the base classifier specified and keeps all the models in a list that is iteratively appended
        features = X_train.copy()
        labels = Y_train.copy()
        if self.model == 'DecisionTree':
            our_model = tree.DecisionTreeClassifier(criterion="entropy")
        elif self.model == 'kNN':
            our_model = neighbors.KNeighborsClassifier()
        elif self.model == 'SVC':
            our_model = SVC(kernel='rbf')
        elif self.model == 'NaiveBayes':
            our_model = GaussianNB()
        elif self.model == 'LogisticRegression':
            our_model = linear_model.LogisticRegression()
        elif self.model == 'Bagging':
            our_model = ensemble.BaggingClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 50), n_estimators=10)
        elif self.model == 'Boosting':
            our_model = ensemble.AdaBoostClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 50), n_estimators=10)
        elif self.model == 'RandomForest':
            our_model = ensemble.RandomForestClassifier(n_estimators = 200, max_features=15, min_samples_split=100)

        our_final_model = list()
        for i in range(len(labels.columns)):
            our_model = our_model.fit(features,labels.iloc[:,i])
            result = our_model.predict(features)
            heading = 'Pred'+str(i+1) #appending the predicted classes of each model to the features, to be used for training in the next iteration
            features[heading] = result
            our_final_model.append(copy.deepcopy(our_model))
        
        self.length_ = len(labels.columns)
        self.our_final_model_ = our_final_model

        return self
    
    def predict_proba(self, features_test):
        features_test = X_test.copy()
        prediction = np.ones((len(features_test), self.length_))
        prediction_proba=np.zeros((len(features_test),2,self.length_))
        for k in range(self.length_):
            each_model=self.our_final_model_[k]
            prediction_proba[:,:,k]=each_model.predict_proba(features_test)
            prediction[:,k]=np.argmax(prediction_proba[:,:,k],axis=1)
            heading = 'Pred'+str(k+1)
            features_test[heading] = prediction[:,k]
            
            
        return prediction_proba
 
    def predict(self, X_test):
        #uses the models found during fit and returns predictions as a matrix
        features_test = X_test.copy()
        prediction = np.ones((len(features_test), self.length_))
        for k in range(self.length_):
            each_model = self.our_final_model_[k]
            prediction[:,k] = each_model.predict(features_test)
            heading = 'Pred'+str(k+1)
            features_test[heading] = prediction[:,k]

        return prediction

In [61]:
cc_model = ClassifierChainImplement('DecisionTree')
cc_model.fit(X_train, y_train)

ClassifierChainImplement(model='DecisionTree')

In [62]:
predict_probabilities_valid=cc_model.predict_proba(X_valid)
predict_probabilities_valid

array([[[0., 0., 1., ..., 0., 0., 1.],
        [1., 1., 0., ..., 1., 1., 0.]],

       [[1., 1., 0., ..., 1., 1., 1.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 1.],
        [1., 1., 0., ..., 1., 1., 0.]],

       ...,

       [[0., 0., 1., ..., 0., 0., 1.],
        [1., 1., 0., ..., 1., 1., 0.]],

       [[1., 1., 1., ..., 0., 0., 1.],
        [0., 0., 0., ..., 1., 1., 0.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [63]:
#prediction on validation set and evaluation metrics
prediction_cc_valid = cc_model.predict(X_valid)
hammingloss_cc_valid = hamming_loss(y_valid,prediction_cc_valid)
print("Hamming loss: " + str(hammingloss_cc_valid))
f1_score_cc_valid = metrics.f1_score(y_valid,prediction_cc_valid,average='macro')
print("Macro-averaged F1 score: " + str(f1_score_cc_valid))
multilabel_confusion_matrix(y_valid,prediction_cc_valid)

Hamming loss: 0.2842384887839433
Macro-averaged F1 score: 0.3893301420541208


array([[[254,  83],
        [ 78,  69]],

       [[163, 114],
        [111,  96]],

       [[186,  94],
        [ 96, 108]],

       [[229,  62],
        [102,  91]],

       [[255,  83],
        [ 83,  63]],

       [[291,  82],
        [ 72,  39]],

       [[342,  58],
        [ 58,  26]],

       [[316,  71],
        [ 71,  26]],

       [[408,  40],
        [ 31,   5]],

       [[380,  51],
        [ 41,  12]],

       [[368,  55],
        [ 45,  16]],

       [[ 28,  88],
        [ 79, 289]],

       [[ 32,  88],
        [ 77, 287]],

       [[471,   7],
        [  6,   0]]], dtype=int64)

In [64]:
accuracy_cc_valid = accuracy_score(y_valid, prediction_cc_valid)
print("Accuracy: " +  str(accuracy_cc_valid))
print(metrics.classification_report(y_valid, prediction_cc_valid))

Accuracy: 0.09917355371900827
              precision    recall  f1-score   support

           0       0.45      0.47      0.46       147
           1       0.46      0.46      0.46       207
           2       0.53      0.53      0.53       204
           3       0.59      0.47      0.53       193
           4       0.43      0.43      0.43       146
           5       0.32      0.35      0.34       111
           6       0.31      0.31      0.31        84
           7       0.27      0.27      0.27        97
           8       0.11      0.14      0.12        36
           9       0.19      0.23      0.21        53
          10       0.23      0.26      0.24        61
          11       0.77      0.79      0.78       368
          12       0.77      0.79      0.78       364
          13       0.00      0.00      0.00         6

   micro avg       0.54      0.54      0.54      2077
   macro avg       0.39      0.39      0.39      2077
weighted avg       0.54      0.54      0.54      2

C:\Users\Arlene John\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
# Check performance with pre-built algorithm from scikit-multilearn
# Reference: https://xang1234.github.io/multi-label/

check_alg2 = ClassifierChain(classifier = tree.DecisionTreeClassifier(), require_dense=None)
tr2 = check_alg2.fit(X_train, y_train)
pred2 = check_alg2.predict(X_test)
pred2_ham = hamming_loss(y_test, pred2)
pred2_f1 = metrics.f1_score(y_test, pred2,average='macro')
acc2_check = accuracy_score(y_test, pred2)
print("Hamming loss: " + str(pred2_ham))
print("Macro-averaged F1 score: " + str(pred2_f1))
print("Accuracy: " + str(acc2_check))

Hamming loss: 0.28364817001180637
Macro-averaged F1 score: 0.37521964751108944
Accuracy: 0.09504132231404959


If we compare the performance of our algorithm (below) and from a pre-built classifier chain (above) from the skmultilearn library, it is possible to observe that the hamming loss and F1 scores are very similar, almost identical, to our classifier chain implementation. Therefore, our Classifier Chain can be validated and it is working as it should.

In [66]:
#prediction on test set and evaluation metrics
prediction_cc_test = cc_model.predict(X_test)
hammingloss_cc_test = hamming_loss(y_test,prediction_cc_test)
print("Hamming loss: " + str(hammingloss_cc_test))
f1_score_cc_test = metrics.f1_score(y_test,prediction_cc_test,average='macro')
print("Macro-averaged F1 score: " + str(f1_score_cc_test))
multilabel_confusion_matrix(y_test,prediction_cc_test)

Hamming loss: 0.282172373081464
Macro-averaged F1 score: 0.3773475829343335


array([[[377, 111],
        [103, 135]],

       [[237, 168],
        [162, 159]],

       [[311, 132],
        [121, 162]],

       [[360, 122],
        [126, 118]],

       [[400, 109],
        [119,  98]],

       [[436, 114],
        [115,  61]],

       [[504, 107],
        [ 81,  34]],

       [[465, 122],
        [102,  37]],

       [[613,  55],
        [ 49,   9]],

       [[591,  60],
        [ 67,   8]],

       [[570,  76],
        [ 70,  10]],

       [[ 52, 142],
        [133, 399]],

       [[ 57, 142],
        [136, 391]],

       [[702,  14],
        [ 10,   0]]], dtype=int64)

In [67]:
accuracy_cc_test = accuracy_score(y_test, prediction_cc_test)
print("Accuracy: " +  str(accuracy_cc_test))
print(metrics.classification_report(y_test, prediction_cc_test))

Accuracy: 0.10606060606060606
              precision    recall  f1-score   support

           0       0.55      0.57      0.56       238
           1       0.49      0.50      0.49       321
           2       0.55      0.57      0.56       283
           3       0.49      0.48      0.49       244
           4       0.47      0.45      0.46       217
           5       0.35      0.35      0.35       176
           6       0.24      0.30      0.27       115
           7       0.23      0.27      0.25       139
           8       0.14      0.16      0.15        58
           9       0.12      0.11      0.11        75
          10       0.12      0.12      0.12        80
          11       0.74      0.75      0.74       532
          12       0.73      0.74      0.74       527
          13       0.00      0.00      0.00        10

   micro avg       0.52      0.54      0.53      3015
   macro avg       0.37      0.38      0.38      3015
weighted avg       0.53      0.54      0.53      3

C:\Users\Arlene John\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
#store evaluation metrics in the dataframe
df2['Classifier Chain (CC)']['Hamming Loss ValidSet'] = hammingloss_cc_valid
df2['Classifier Chain (CC)']['F1 Score ValidSet'] = f1_score_cc_valid
df2['Classifier Chain (CC)']['Accuracy ValidSet'] = accuracy_cc_valid
df2['Classifier Chain (CC)']['Hamming Loss TestSet'] = hammingloss_cc_test
df2['Classifier Chain (CC)']['F1 Score TestSet'] = f1_score_cc_test
df2['Classifier Chain (CC)']['Accuracy TestSet'] = accuracy_cc_test

In [69]:
df2

,Binary Relavance without Undersampling,Binary Relavance with Undersampling,Optimal Binary Relavance without Undersampling (F1 score),Optimal Binary Relavance with Undersampling (F1 score),Classifier Chain (CC),Optimal Classifier Chains (F1 score),Optimal Changed Chain Sequence CC
Hamming Loss ValidSet,0.266972,0.423259,0.287338,0.338400,0.284238,0.0,0.0
F1 Score ValidSet,0.407905,0.407460,0.481440,0.480174,0.389330,0.0,0.0
Accuracy ValidSet,0.037190,0.000000,0.103306,0.059917,0.099174,0.0,0.0
Hamming Loss TestSet,0.273022,0.423849,0.293388,0.349272,0.282172,0.0,0.0
F1 Score TestSet,0.398105,0.395571,0.445150,0.451297,0.377348,0.0,0.0
Accuracy TestSet,0.042700,0.002755,0.097796,0.070248,0.106061,0.0,0.0


## Task 5: Evaluate the Performance of the Classifier Chains Algorithm

In [70]:
#grid search for best base classifier optimized for f1-score
param_grid = [{'model':('DecisionTree','kNN','SVC','NaiveBayes','LogisticRegression','Bagging', 'Boosting', 'RandomForest')}]
best_model1 = GridSearchCV(ClassifierChainImplement(),param_grid=param_grid,cv=10,n_jobs=-1,scoring='f1_macro')
best_model1.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=ClassifierChainImplement(model='DecisionTree'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'model': ('DecisionTree', 'kNN', 'SVC', 'NaiveBayes',
                                    'LogisticRegression', 'Bagging', 'Boosting',
                                    'RandomForest')}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [71]:
print("Best parameters set found on development set:")
print(best_model1.best_params_)
print(best_model1.best_score_)

Best parameters set found on development set:
{'model': 'NaiveBayes'}
0.4332779269901894


In [72]:
#prediction on validation set and evaluation metrics
prediction_cc_valid_opt = best_model1.predict(X_valid)
hammingloss_cc_valid_opt = hamming_loss(y_valid,prediction_cc_valid_opt)
f1_score_cc_valid_opt = metrics.f1_score(y_valid,prediction_cc_valid_opt,average='macro')
accuracy_cc_valid_opt = accuracy_score(y_valid, prediction_cc_valid_opt)
print("Hamming loss: " + str(hammingloss_cc_valid_opt))
print("Macro-averaged F1 score: " + str(f1_score_cc_valid_opt))
print("Accuracy: " +  str(accuracy_cc_valid_opt))
#multilabel_confusion_matrix(y_valid,prediction_cc_valid_opt)

Hamming loss: 0.2989964580873672
Macro-averaged F1 score: 0.4671236690962089
Accuracy: 0.09297520661157024


In [73]:
print("Accuracy: " +  str(accuracy_cc_valid_opt))
print(metrics.classification_report(y_valid, prediction_cc_valid_opt))

Accuracy: 0.09297520661157024
              precision    recall  f1-score   support

           0       0.55      0.61      0.58       147
           1       0.54      0.71      0.61       207
           2       0.65      0.76      0.70       204
           3       0.66      0.58      0.61       193
           4       0.54      0.54      0.54       146
           5       0.39      0.62      0.48       111
           6       0.27      0.52      0.35        84
           7       0.31      0.54      0.40        97
           8       0.12      0.47      0.20        36
           9       0.17      0.43      0.25        53
          10       0.21      0.48      0.29        61
          11       0.87      0.63      0.73       368
          12       0.86      0.63      0.73       364
          13       0.05      0.50      0.09         6

   micro avg       0.51      0.61      0.56      2077
   macro avg       0.44      0.57      0.47      2077
weighted avg       0.62      0.61      0.60      2

In [74]:
#prediction on test set and evaluation metrics
prediction_cc_test_opt = best_model1.predict(X_test)
hammingloss_cc_test_opt = hamming_loss(y_test,prediction_cc_test_opt)
f1_score_cc_test_opt = metrics.f1_score(y_test,prediction_cc_test_opt,average='macro')
accuracy_cc_test_opt = accuracy_score(y_test, prediction_cc_test_opt)
print("Hamming loss: " + str(hammingloss_cc_test_opt))
print("Macro-averaged F1 score: " + str(f1_score_cc_test_opt))
print("Accuracy: " +  str(accuracy_cc_test_opt))
multilabel_confusion_matrix(y_test,prediction_cc_test_opt)

Hamming loss: 0.30794962613144433
Macro-averaged F1 score: 0.43883754795097657
Accuracy: 0.09228650137741047


array([[[392,  96],
        [ 80, 158]],

       [[231, 174],
        [121, 200]],

       [[293, 150],
        [ 69, 214]],

       [[352, 130],
        [104, 140]],

       [[400, 109],
        [100, 117]],

       [[386, 164],
        [ 80,  96]],

       [[431, 180],
        [ 55,  60]],

       [[419, 168],
        [ 73,  66]],

       [[511, 157],
        [ 41,  17]],

       [[500, 151],
        [ 43,  32]],

       [[474, 172],
        [ 53,  27]],

       [[ 99,  95],
        [190, 342]],

       [[ 99, 100],
        [190, 337]],

       [[638,  78],
        [  7,   3]]], dtype=int64)

In [75]:
print("Accuracy: " +  str(accuracy_cc_test_opt))
print(metrics.classification_report(y_test, prediction_cc_test_opt))

Accuracy: 0.09228650137741047
              precision    recall  f1-score   support

           0       0.62      0.66      0.64       238
           1       0.53      0.62      0.58       321
           2       0.59      0.76      0.66       283
           3       0.52      0.57      0.54       244
           4       0.52      0.54      0.53       217
           5       0.37      0.55      0.44       176
           6       0.25      0.52      0.34       115
           7       0.28      0.47      0.35       139
           8       0.10      0.29      0.15        58
           9       0.17      0.43      0.25        75
          10       0.14      0.34      0.19        80
          11       0.78      0.64      0.71       532
          12       0.77      0.64      0.70       527
          13       0.04      0.30      0.07        10

   micro avg       0.48      0.60      0.54      3015
   macro avg       0.41      0.52      0.44      3015
weighted avg       0.57      0.60      0.57      3

In [76]:
#store evaluation metrics in the dataframe
df2['Optimal Classifier Chains (F1 score)']['Hamming Loss ValidSet'] = hammingloss_cc_valid_opt
df2['Optimal Classifier Chains (F1 score)']['F1 Score ValidSet'] = f1_score_cc_valid_opt
df2['Optimal Classifier Chains (F1 score)']['Accuracy ValidSet'] = accuracy_cc_valid_opt
df2['Optimal Classifier Chains (F1 score)']['Hamming Loss TestSet'] = hammingloss_cc_test_opt
df2['Optimal Classifier Chains (F1 score)']['F1 Score TestSet'] = f1_score_cc_test_opt
df2['Optimal Classifier Chains (F1 score)']['Accuracy TestSet'] = accuracy_cc_test_opt

### Comparison between Binary Relevance and Classifier Chains

In [77]:
df2

,Binary Relavance without Undersampling,Binary Relavance with Undersampling,Optimal Binary Relavance without Undersampling (F1 score),Optimal Binary Relavance with Undersampling (F1 score),Classifier Chain (CC),Optimal Classifier Chains (F1 score),Optimal Changed Chain Sequence CC
Hamming Loss ValidSet,0.266972,0.423259,0.287338,0.338400,0.284238,0.298996,0.0
F1 Score ValidSet,0.407905,0.407460,0.481440,0.480174,0.389330,0.467124,0.0
Accuracy ValidSet,0.037190,0.000000,0.103306,0.059917,0.099174,0.092975,0.0
Hamming Loss TestSet,0.273022,0.423849,0.293388,0.349272,0.282172,0.307950,0.0
F1 Score TestSet,0.398105,0.395571,0.445150,0.451297,0.377348,0.438838,0.0
Accuracy TestSet,0.042700,0.002755,0.097796,0.070248,0.106061,0.092287,0.0


Base Classifiers:

Binary Relavance with and without undersampling: Decision Tree
 <br />
Optimised Model with F1 Score for Binary relevance classifiers: SVC with undersampling, Naive Bayes without undersampling
 <br />
Optimised Model with Accuracy: kNN with undersampling, kNN without undersampling <br />
Optimal Classifier Chains: Naive Bayes <br/>


As can be observed from the dataframe presented above, the results for the Classifier Chains did not improve the classification process greatly when compared with the binary relevance algorithms. The hamming loss of the the classifier chain is lower than that of the binary relevance classifier. However, the optimal classifier chain which was optimized for F1-score shows an increase in F1-score and accuracy when compared against the normal classifier chain, but performs poorly compared to the binary relevance method in all three metrics considered. The Classifier Chains is theoretically an improvement on the Binary Relevance algorithm because it also considers label dependencies. However, in this case, its performance was not better. It was still a solid performance, with reasonable values for Hamming loss and F1 score. The reasons why the Classifier Chains did not perform could be due to the order of the labels and the chain sequence. If we optimize the chain sequence, the Classifier Chain might exhibit superior performance. Comparing the optimal models of Binary Relevance (SVC with undersampling) and Classifier Chains (Naive Bayes), the Classifier Chains did not outperform the Binary Relevance method.

### Changes in Chain Sequence 

To investigate if the chain sequence can be improved to improve its performance, we have ordered the labels based on their scores from the Binary Relevance algorithm.

In [78]:
#Reorder the labels based in the decreasing order of accuracy for different labels in binary relevance classifiers (new_column determined in Task 1)
new_column=new_column.T[0]
y_train_new=y_train.iloc[:,new_column]
y_valid_new=y_valid.iloc[:,new_column]
y_test_new=y_test.iloc[:,new_column]

In [79]:
#grid search for best base classifier optimized for f1-score
param_grid = [{'model':('DecisionTree','kNN','SVC','NaiveBayes','LogisticRegression','Bagging', 'Boosting', 'RandomForest')}]
best_model_cc_cs = GridSearchCV(ClassifierChainImplement(),param_grid=param_grid,cv=10,n_jobs=-1,scoring='f1_macro')
best_model_cc_cs.fit(X_train,y_train_new)

GridSearchCV(cv=10, error_score=nan,
             estimator=ClassifierChainImplement(model='DecisionTree'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'model': ('DecisionTree', 'kNN', 'SVC', 'NaiveBayes',
                                    'LogisticRegression', 'Bagging', 'Boosting',
                                    'RandomForest')}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [80]:
print("Best parameters set found on development set:")
print(best_model_cc_cs.best_params_)
print(best_model_cc_cs.best_score_)

Best parameters set found on development set:
{'model': 'NaiveBayes'}
0.4404358941407335


In [81]:
#prediction on validation set and test set and evaluation metrics
print(' ')
print('Sequence Chain Optimised')
print(' ')
print(' ')
print('Validation Set:')
print(' ')
prediction_cc_valid_cs = best_model_cc_cs.predict(X_valid)
hammingloss_cc_valid_cs = hamming_loss(y_valid_new,prediction_cc_valid_cs)
print("Hamming loss: " + str(hammingloss_cc_valid_cs))
f1_score_cc_valid_cs = metrics.f1_score(y_valid_new,prediction_cc_valid_cs,average='macro')
print("Macro-averaged F1 score: " + str(f1_score_cc_valid_cs))
multilabel_confusion_matrix(y_valid_new,prediction_cc_valid_cs)
accuracy_cc_valid_cs = accuracy_score(y_valid_new, prediction_cc_valid_cs)
print("Accuracy: " +  str(accuracy_cc_valid_cs))
print(' ')
print('Test Set:')
print(' ')
prediction_cc_test_cs = best_model_cc_cs.predict(X_test)
hammingloss_cc_test_cs = hamming_loss(y_test_new, prediction_cc_test_cs)
print("Hamming loss: " + str(hammingloss_cc_test_cs))
f1_score_cc_test_cs = metrics.f1_score(y_test_new, prediction_cc_test_cs,average='macro')
print("Macro-averaged F1 score: " + str(f1_score_cc_test_cs))
multilabel_confusion_matrix(y_test_new,prediction_cc_test_cs)
accuracy_cc_test_cs = accuracy_score(y_test_new, prediction_cc_test_cs)
print("Accuracy: " +  str(accuracy_cc_test_cs))



 
Sequence Chain Optimised
 
 
Validation Set:
 
Hamming loss: 0.2953069657615112
Macro-averaged F1 score: 0.4784839761487472
Accuracy: 0.10743801652892562
 
Test Set:
 
Hamming loss: 0.30509641873278237
Macro-averaged F1 score: 0.43778615836460055
Accuracy: 0.10055096418732783


In [82]:
#store evaluation metrics in the dataframe
df2['Optimal Changed Chain Sequence CC']['Hamming Loss ValidSet'] = hammingloss_cc_valid_cs
df2['Optimal Changed Chain Sequence CC']['F1 Score ValidSet'] = f1_score_cc_valid_cs
df2['Optimal Changed Chain Sequence CC']['Accuracy ValidSet'] = accuracy_cc_valid_cs

df2['Optimal Changed Chain Sequence CC']['Hamming Loss TestSet'] = hammingloss_cc_test_cs
df2['Optimal Changed Chain Sequence CC']['F1 Score TestSet'] = f1_score_cc_test_cs
df2['Optimal Changed Chain Sequence CC']['Accuracy TestSet'] = accuracy_cc_test_cs


In [83]:
df2

,Binary Relavance without Undersampling,Binary Relavance with Undersampling,Optimal Binary Relavance without Undersampling (F1 score),Optimal Binary Relavance with Undersampling (F1 score),Classifier Chain (CC),Optimal Classifier Chains (F1 score),Optimal Changed Chain Sequence CC
Hamming Loss ValidSet,0.266972,0.423259,0.287338,0.338400,0.284238,0.298996,0.295307
F1 Score ValidSet,0.407905,0.407460,0.481440,0.480174,0.389330,0.467124,0.478484
Accuracy ValidSet,0.037190,0.000000,0.103306,0.059917,0.099174,0.092975,0.107438
Hamming Loss TestSet,0.273022,0.423849,0.293388,0.349272,0.282172,0.307950,0.305096
F1 Score TestSet,0.398105,0.395571,0.445150,0.451297,0.377348,0.438838,0.437786
Accuracy TestSet,0.042700,0.002755,0.097796,0.070248,0.106061,0.092287,0.100551


Base Classifiers:

Binary Relavance with and without undersampling: Decision Tree
 <br />
Optimised Model with F1 Score for Binary relevance classifiers: SVC with undersampling, Naive Bayes without undersampling
 <br />
Optimised Model with Accuracy: kNN with undersampling, kNN without undersampling <br />
Optimal Classifier Chains: Naive Bayes <br/>
Optimal Chain Sequenced CC: Naive Bayes

Changing the chaining sequence and finding the optimal base classifier shows a slightly better performance than the simple optimized chain classifier in terms of accuracy, F1 score and hamming loss in the validation set. However, in the test set, the original classifier chain has a better F1 score. <br/>
However, when compared against the best performing binary relevance classifier, the optimized changed chain sequence-based classifier chain still performs poorly.

## Task 6: Reflect on the Performance of the Different Models Evaluated

The models approached in this assignment were the Binary Relevance (BR) and the Classifier Chains (CC). BR treats the multilabel classification by trying to learn a different model for each label. In this assignment, we also applied undersampling to try to solve the problem of imbalanced classes. CC is a more complex learner than the Binary Relevance classifier as it considers label dependencies and includes the prediction of labels as input for new predictions. CCs get computationally more expensive for each subsequent label classifier as the number of feature increase. Theoretically, it is expected that the CC performs better than the BR for the fact it considers label dependencies. However, both algorithms exhibited similar performance, with the Binary relevance classifier being slightly better. This could be attributed to the non-optimal chain sequence when building the CC. To investigate this, we tried a simple binary relevance performance-based chain sequence optimization method. It showed improvements when compared against a normal classifier chain but still did not outperform the Binary Relevance classifier for this dataset. Undersampling the dataset was helpful to address the imbalanced classes. However, the undersampling showed a drop in performance for the metrics considered here. This could be due to the loss of information due to undersampling and the subsequent loss in significant training examples. Thus, the Binary Relevance method proved to be an effective method in tackling this problem, since it is comparatively less complex and computationally less expensive. It is worth noting that neither of the algorithms exhibited outstanding results and optimization methods could be applied to try to improve them. 